In [1]:
import os
from os.path import isdir, join
from pathlib import Path
import pandas as pd

# Math
import numpy as np
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
import librosa

from sklearn.decomposition import PCA

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd
import librosa.display

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import pandas as pd

%matplotlib inline

In [2]:
train_audio_path="20sp_lecture"

dirs = [f for f in os.listdir(train_audio_path) if isdir(join(train_audio_path, f))]
dirs.sort()
dirs.remove('transcript')
print('Number of speakers: ' + str(len(dirs)))

Number of speakers: 20


In [3]:
# Calculate
number_of_recordings = []
for direct in dirs:
    waves = [f for f in os.listdir(join(train_audio_path, direct)) if f.endswith('.wav')]
    number_of_recordings.append(len(waves))
print(number_of_recordings)

[83, 328, 216, 158, 131, 161, 148, 97, 224, 164, 153, 216, 162, 76, 48, 87, 95, 87, 159, 206]


In [14]:
# Plot
data = [go.Histogram(x=dirs, y=number_of_recordings)]
trace = go.Bar(
    x=dirs,
    y=number_of_recordings,
    marker=dict(color = number_of_recordings, showscale=True
    ),
)
layout = go.Layout(
    title='Number of recordings in given label',
    xaxis = dict(title='Speakers'),
    yaxis = dict(title='Number of recordings')
)
py.iplot(go.Figure(data=[trace], layout=layout))

In [5]:
num_of_shorter = 0
for direct in dirs:
    waves = [f for f in os.listdir(join(train_audio_path, direct)) if f.endswith('.wav')]
    for wav in waves:
        sample_rate, samples = wavfile.read(train_audio_path + '/' + direct + '/' + wav)
        if samples.shape[0] < sample_rate:
            num_of_shorter += 1
print('Number of recordings shorter than 1 second: ' + str(num_of_shorter))

C:\Users\LENOVO\AppData\Local\Continuum\miniconda3\lib\site-packages\scipy\io\wavfile.py:273: WavFileWarning:

Chunk (non-data) not understood, skipping it.



Number of recordings shorter than 1 second: 83


In [6]:
def custom_fft(y, fs):
    T = 1.0 / fs
    N = y.shape[0]
    yf = fft(y)
    xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
    vals = 2.0/N * np.abs(yf[0:N//2])  # FFT is simmetrical, so we take just the first half
    # FFT is also complex, to we take just the real part (abs)
    return xf, vals

In [7]:
max_samples = 0
for direct in dirs:
    waves = [f for f in os.listdir(join(train_audio_path, direct)) if f.endswith('.wav')]
    for wav in waves:
        sample_rate, samples = wavfile.read(train_audio_path + '/' + direct + '/' + wav)
        max_samples = samples.shape[0] if samples.shape[0] > max_samples else max_samples
max_samples

806457

In [8]:
fft_all = []
names = []
iterator = 0
for direct in dirs:
    waves = [f for f in os.listdir(join(train_audio_path, direct)) if f.endswith('.wav')]
    print(direct)
    for wav in waves:
        sample_rate, samples = wavfile.read(train_audio_path + '/' + direct + '/' + wav)
        if samples.shape[0] != max_samples:
            delta = max_samples - samples.shape[0]
            print(delta, sample_rate)
            samples = np.append(samples, np.zeros((delta, )))
#         x, val = custom_fft(samples, sample_rate)
        x, val = custom_fft(samples, max_samples)
        print('len x and val ', len(x), len(val))
        fft_all.append(val)
        names.append(direct + '/' + wav)
        
        iterator += 1
        if iterator == 3:
            break
    break

fft_all = np.array(fft_all)

f01
762310 16000
len x and val  403228 403228
675094 16000
len x and val  403228 403228
787076 16000
len x and val  403228 403228


In [9]:
# Normalization
# temp = []
# for obj in fft_all:
fft_all = (fft_all - np.mean(fft_all, axis=0)) / np.std(fft_all, axis=0)
#     temp.append((obj - np.mean(obj, axis=0)) / np.std(obj, axis=0))
# print(temp)

In [10]:
fft_all

array([[ 0.6477962 ,  0.64936911,  0.6575709 , ..., -0.79354384,
        -0.76267945, -0.75230396],
       [ 0.76480299,  0.76331352,  0.75551173, ...,  1.41053527,
         1.41271746,  1.41322915],
       [-1.41259919, -1.41268263, -1.41308263, ..., -0.61699143,
        -0.65003801, -0.66092519]])

In [12]:
# Dim reduction
pca = PCA(n_components=3)
fft_all = pca.fit_transform(fft_all)

def interactive_3d_plot(data, names):
    scatt = go.Scatter3d(x=data[:, 0], y=data[:, 1], z=data[:, 2], mode='markers', text=names)
    data = go.Data([scatt])
    layout = go.Layout(title="Anomaly detection")
    figure = go.Figure(data=data, layout=layout)
    py.iplot(figure)

In [13]:
interactive_3d_plot(fft_all, names)

C:\Users\LENOVO\AppData\Local\Continuum\miniconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:40: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.


